# **Simple hybrid recommendation system**

**Гибридная РС** — это особый тип рекомендательной системы, который представляет собой комбинацию из нескольких методов. Обычно это комбинация контентного подхода и коллаборативной фильтрации. Такое сочетание может помочь преодолеть недостатки, с которыми мы сталкиваемся при использовании этих методов по отдельности, а также в некоторых случаях может быть более эффективным.

Гибридные подходы к рекомендательным системам можно реализовать по-разному, например используя комбинацию из рекомендаций, полученных обеими системами, или используя взвешенную сумму предсказанных рейтингов.

![image.png](hybrid_recommendations.png)

Попробуем создать рекомендательную систему с помощью гибридного подхода.

## Загрузка инструментов

Разумеется, можно комбинировать различные подходы самостоятельно, однако для удобства уже реализован модуль LightFM.

In [3]:
!pip install lightfm -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


Импортируем нужные нам функции из этой библиотеки. На этом этапе сразу же загрузим инструменты оценки модели:

In [20]:
import os
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k

## Загрузка и обработка данных

Работать мы будем с датасетом [goodreads_book](https://lms-cdn.skillfactory.ru/assets/courseware/v1/c977535583bf2f85a2d15617e672d8f4/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/Gooddreadbooks.zip).

*Goodreads* — это сайт, на котором люди могут добавлять книги в каталоги, искать их, изучать аннотации и отзывы. Пользователи также могут создавать сообщества, в которых они рекомендуют друг другу различную литературу, ведут блоги и устраивают обсуждения.

Подгрузим все файлы, относящиеся к этому набору данных:

In [10]:
ratings = pd.read_csv('data/ratings.csv')     # Поставленные оценки
print('ratings df shape:', ratings.shape)
books = pd.read_csv('data/books.csv')         # Информация о книгах
print('books df shape:', books.shape)
tags = pd.read_csv('data/tags.csv')           # Информация о тегах
print('tags df shape:', tags.shape)
book_tags = pd.read_csv('data/book_tags.csv') # Книги с тегами
print('book_tags df shape:', book_tags.shape)

ratings df shape: (981756, 3)
books df shape: (10000, 23)
tags df shape: (334, 2)
book_tags df shape: (999912, 3)


In [11]:
display(ratings.head(3))
display(books.head(3))
display(tags.head(3))
display(book_tags.head(3))

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5


,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...


,tag_id,tag_name
0,509,19th-century
1,923,20th-century
2,941,21st-century


,goodreads_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173


Сначала посмотрим на набор данных `books`: в этих данных есть обычный *id* книги, а есть *id* книги в системе *Goodreads* — этот *id* отображён в признаке `goodreads_book_id`. В других данных (`book_tags`) указан только *id* книги в системе *Goodreads*, поэтому нам необходимо добавить туда обычный *id*.

Добавим в набор данных `book_tags` признак с обычным *id* книги, используя соответствие обычного *id* и *id* в системе *Goodreads*.

In [12]:
dict_map = dict(zip(books.goodreads_book_id,books.book_id))
book_tags['book_id'] = book_tags.goodreads_book_id.apply(
    lambda x: dict_map[x]
    )
# Пример
book_tags[book_tags['goodreads_book_id']==5][:1]

,goodreads_book_id,tag_id,count,book_id
300,5,11557,40087,18


Отфильтруем данные таким образом, чтобы в наборе данных `book_tags` остались только те строки, в которых находятся теги, информация о которых есть в наборе данных `tags`.


In [13]:
print('before book_tags shape:', book_tags.shape)
book_tags = book_tags[book_tags.tag_id.isin(tags.tag_id)]
print('after book_tags shape:', book_tags.shape)

before book_tags shape: (999912, 4)
after book_tags shape: (300738, 4)


Отлично, мы подготовили информацию о тегах книг — это будет метаинформацией для построения рекомендательной системы. Теперь нам необходимо подготовить данные о взаимодействии пользователей и книг. Для этого нам понадобится файл *ratings*.

Оба набора данных (и про взаимодействия, и про метаинформацию) необходимо преобразовать в разрежённые матрицы. Это можно сделать с помощью специальной функции из модуля `scipy`:

    csr_matrix

Нам важно преобразовать данные в специальный формат, в котором хранятся разрежённые матрицы — будем использовать формат Compressed Sparse Row (CSR), подразумевающий подсчёт кумулятивной суммы количества элементов в строке вместо индексов строк.

Осуществляем преобразование следующим образом:

In [14]:
# Передаём в качестве аргументов в функцию выставленный рейтинг
# (это будут значения матрицы), а также id пользователя и id книги
#  (это будут индексы для строк и столбцов матрицы)
ratings_matrix = csr_matrix(
    (ratings.rating,(ratings.user_id, ratings.book_id))
    )

Теперь нам необходимо составить матрицу с метаданными. В качестве индексов будут выступать id книги и id тега, и если у этой книги есть рассматриваемый тег, то на пересечении соответствующих строки и столбца будет выставлена единица.

In [15]:
meta_matrix  = csr_matrix(
    ([1]*len(book_tags), (book_tags.book_id, book_tags.tag_id))
    )

Проверим, что всё получилось правильно. Посчитаем среднее арифметическое значений разрежённой матрицы с рейтингами.

In [16]:
round(ratings_matrix.mean(), 3)

0.007

## Моделирование

Отлично, данные подготовлены — теперь настало время определить модель, которую мы будем использовать. Сделаем это следующим образом:

In [17]:
model = LightFM(
    loss='warp-kos',    # Определяем функцию потерь
    random_state=42,    # Фиксируем случайное разбиение
    learning_rate=0.05, # Темп обучения
    no_components=100   # Размерность вектора для представления данных в модели
)

Разобьём данные на обучающую и тестовую выборки:

In [18]:
train, test = random_train_test_split(
    ratings_matrix,      # Общая выборка
    test_percentage=0.2, # Размер тестовой выборки
    random_state=42      # Генератор случайных чисел
)

Теперь обучим модель на наших данных о взаимодействии, также используя метаданные о книгах. Для этого воспользуемся методом fit(). В этот метод передадим обучающую выборку, признаки товаров — item_features, количество эпох обучения (сколько раз мы будем показывать модели исходный датасет, чтобы она лучше выучила данные) — epochs, а также параметр verbose для отслеживания процесса обучения:

In [19]:
model = model.fit(
    train,                     # Обучающая выборка
    item_features=meta_matrix, # Признаки товаров
    epochs=10,                 # Количество эпох
    verbose=True,              # Отображение обучения
    num_threads=os.cpu_count() # Число потоков
)

Epoch: 100%|██████████| 10/10 [09:19<00:00, 55.94s/it]


Оценим качество полученной модели с помощью функции precision_at_k, передав в неё три аргумента: модель, тестовые данные и обозначение метаданных (`item_features = meta_matrix`).

>**Примечание.** Процесс расчёта метрик рекомендательной системы также является довольно затратным по времени. Для ускорения этого процесса можно передать параметр num_threads, чтобы указать количество потоков процессора, используемых для вычислений.

In [23]:
%%time
prec_score = precision_at_k(
    model, test,
    item_features=meta_matrix,
    num_threads=os.cpu_count()
).mean()

print(round(prec_score, 4))

0.0237
CPU times: user 26min 50s, sys: 1.99 s, total: 26min 52s
Wall time: 15min 46s


У нас получился не слишком высокий, но довольно неплохой результат. Чтобы его улучшить, можно попробовать следующее:

- Поработать над предобработкой данных, добавив в них дополнительную информацию о товарах. Также можно попробовать воспользоваться иным способом создания разреженной матрицы, например, форматом coo_matrix() или csc_matrix(), которые также входят в библиотеку scipy.
- Поиграться с параметрами модели LightFM — поуправлять темпом обучения (learning_rate), размерностью вектора для представления (no_components), количеством эпох обучения (epochs) и функцией потерь (loss).